# Multiple Regression

Multiple regression is the natural extension of linear regression such that we fit a single value from a collection of parameters.

**NOTE: in the last notebooks I went against the book and swapped alpha ($\alpha$) and beta ($\beta$) in usage, as I believed it to be the notation I was already familiar with. This was probably wrong, and in any case will only get increasingly hard to maintain. From here on out
In the linear regression case: 
$$ y_i = \beta x_i + \alpha + \epsilon $$

In the multiple regression case:

$$ y_i = \beta_0 x_{i0} + ... + \beta_n x_{in} + \alpha + \epsilon $$

where we learn $n + 1$ parameters for $n$ features (+1 = beta/bias). In this context, $x$ is a vector

### The Model in Code

For simplicity, we combine the alpha term into the beta vector and transform our inputs to include a constant term: `[x_0 ... x_n]` becomes `[1, x_0 ... x_n]` and our `beta` vector is really `[alpha, beta_0 ... beta_n]` Prediction is straight-forward

In [1]:
# Awful hack to import past chapter modules
import sys
sys.path.insert(0, "../")
import tqdm, random
from linalg import dot, Vector, vector_mean, vector_sum, scalar_multiply, subtract, add
from probability import normal_cdf
from simple_linear_regression import total_sum_of_squares
from gradient_descent import gradient_step
from typing import List, TypeVar, Callable, Tuple
from stats import median, standard_deviation

In [2]:
def predict(x: Vector, beta: Vector) -> float:
    """
    Given values for x and parameters beta, give a prediction for y. This assumes any bias transform 
    [1, x_0 ... x_n] has already been performed on x, beta
    """
    return dot(x, beta)
    

### Further Assumptions of the Least Squares Model

Minimizing squared-error in multiple regression makes some assumptions about our data that are important to keep in mind, as they aren't always known to be true:

1. Features in $x$ are *linearly independent*, i.e. no single feature in $x$ can be described as a linear combination of any others. A least-squares solution cannot always be determined when this is not true. This often requires careful data cleaning to verify.

2. The columns of $x$ must be uncorrelated with the errors $\epsilon$. Otherwise, our model will be systematically wrong. An example:
  - Consider the relationship between number of friends in a social network ($friends$), number of hours spent working ($work hours$), and number of minutes ($minutes$) spent on the social networks site.
    - Lets say the true relationship is defined as $minutes  = \beta_1 friends + \beta_2 work hours + \epsilon$
    - Lets also say that $work hours$ is correlated with $friends$, i.e. $friends ~ k * work hours + \epsilon_2$
  - If we are trying to fit regression to $minutes = \beta_1 friends + \epsilon$, we will systematically under-estimate $\beta_1$: our predictions will necessarily be less correct for users with *many* friends because the error terms increase as the number of friends increase, due in this case to the hidden relationship with work hours
  - This is known as [*heteroskedacity*](http://www.statsmakemecry.com/smmctheblog/confusing-stats-terms-explained-heteroscedasticity-heteroske.html), and can be explained visually here:
    - ![heteroskedatic data](https://upload.wikimedia.org/wikipedia/en/thumb/5/5d/Hsked_residual_compare.svg/1024px-Hsked_residual_compare.svg.png)
    
  - When data is *heteroskedatic* we get a biased estimate of $\beta$ with respect to the feature(s) correlated with $\epsilon$

### Fitting the Model

We will again fit by minimizing mean squared error


In [3]:
def error(x: Vector, y: float, beta: Vector) -> float:
    """
    with a prediction defined by x * beta, return the error w.r.t the true value y
    """
    return predict(x, beta) - y


def squared_error(x: Vector, y: float, beta: Vector) -> float:
    """
    with a prediction defined by x * beta, return the squared error w.r.t the true value y
    """
    return error(x, y, beta) ** 2


def sq_error_gradient(x: Vector, y: float, beta: Vector) -> Vector:
    """
    For each input in x, return the gradient with respect to that input for a squared-error loss
    """
    err: float = error(x, y, beta)
    return [2 * err * x_i for x_i in x]

def _grad_step(beta: Vector, grad: Vector, lr: float = 10**-3) -> Vector:
    # repeating this to check my knowledge of the gradient update
    update: Vector = scalar_multiply(-1 * lr, grad)
    return vector_sum([beta, update])

x = [1, 2, 3]
y = 30
beta = [4, 4, 4]
assert error(x, y, beta) == -6  # (4 + 8 + 12) - 30 = -6
assert squared_error(x, y, beta) == 36  # ((4 + 8 + 12) - 30)**2 = 36
assert sq_error_gradient(x, y, beta) == [-12, -24, -36]  # -6 * 2 * 1 ...
assert _grad_step(beta, sq_error_gradient(x, y, beta), lr=.01) == [4.12, 4.24, 4.36]
assert gradient_step(beta, sq_error_gradient(x, y, beta), -1 * .01) == [4.12, 4.24, 4.36]

In [4]:
def least_squares_fit(xs: List[Vector], ys: List[Vector], lr:float = 10**-3, num_steps: int = 1000, batch_size: int = 1) -> Vector:
    """
    For the given inputs (`xs`) and outputs (`ys`), find the parameters (`beta`) that provide the best fit 
    via multiple regression. Performs `num_steps` gradient descent steps with batch sizes of `batch_size` 
    and a learning rate of `lr`.
    """
    # paired = list(zip(xs, ys))
    # random.shuffle(paired)  doing this seems correct and exerimentally valid, but gets slightly worse results than the
    # book. Maybe the book has well-selected hyper parameters?
    # xs, ys = zip(*paired)
    beta = [random.random() for r in range(0, len(xs[0]))]
    for _ in tqdm.trange(num_steps, desc="least squares fit"):
        for start in range(0, len(xs), batch_size):
            # prepare a batch
            batch_xs = xs[start:start + batch_size]
            batch_ys = ys[start:start + batch_size]
            # compute an average gradient across the batch
            grads = [sq_error_gradient(x, y, beta) for (x, y) in zip(batch_xs, batch_ys)]
            avg_grad: Vector = vector_mean(grads)
            # update beta according to the gradient
            beta = gradient_step(beta, avg_grad, lr * -1)  # by negative one b/c we are minimizing
    return beta


            
            
            

    

In [5]:
# stolen from: https://github.com/joelgrus/data-science-from-scratch/blob/master/scratch/multiple_regression.py
# [1 for bias, num friends, work hours per day, has PhD]
inputs: List[List[float]] = [[1.,49,4,0],[1,41,9,0],[1,40,8,0],[1,25,6,0],[1,21,1,0],[1,21,0,0],[1,19,3,0],[1,19,0,0],[1,18,9,0],[1,18,8,0],[1,16,4,0],[1,15,3,0],[1,15,0,0],[1,15,2,0],[1,15,7,0],[1,14,0,0],[1,14,1,0],[1,13,1,0],[1,13,7,0],[1,13,4,0],[1,13,2,0],[1,12,5,0],[1,12,0,0],[1,11,9,0],[1,10,9,0],[1,10,1,0],[1,10,1,0],[1,10,7,0],[1,10,9,0],[1,10,1,0],[1,10,6,0],[1,10,6,0],[1,10,8,0],[1,10,10,0],[1,10,6,0],[1,10,0,0],[1,10,5,0],[1,10,3,0],[1,10,4,0],[1,9,9,0],[1,9,9,0],[1,9,0,0],[1,9,0,0],[1,9,6,0],[1,9,10,0],[1,9,8,0],[1,9,5,0],[1,9,2,0],[1,9,9,0],[1,9,10,0],[1,9,7,0],[1,9,2,0],[1,9,0,0],[1,9,4,0],[1,9,6,0],[1,9,4,0],[1,9,7,0],[1,8,3,0],[1,8,2,0],[1,8,4,0],[1,8,9,0],[1,8,2,0],[1,8,3,0],[1,8,5,0],[1,8,8,0],[1,8,0,0],[1,8,9,0],[1,8,10,0],[1,8,5,0],[1,8,5,0],[1,7,5,0],[1,7,5,0],[1,7,0,0],[1,7,2,0],[1,7,8,0],[1,7,10,0],[1,7,5,0],[1,7,3,0],[1,7,3,0],[1,7,6,0],[1,7,7,0],[1,7,7,0],[1,7,9,0],[1,7,3,0],[1,7,8,0],[1,6,4,0],[1,6,6,0],[1,6,4,0],[1,6,9,0],[1,6,0,0],[1,6,1,0],[1,6,4,0],[1,6,1,0],[1,6,0,0],[1,6,7,0],[1,6,0,0],[1,6,8,0],[1,6,4,0],[1,6,2,1],[1,6,1,1],[1,6,3,1],[1,6,6,1],[1,6,4,1],[1,6,4,1],[1,6,1,1],[1,6,3,1],[1,6,4,1],[1,5,1,1],[1,5,9,1],[1,5,4,1],[1,5,6,1],[1,5,4,1],[1,5,4,1],[1,5,10,1],[1,5,5,1],[1,5,2,1],[1,5,4,1],[1,5,4,1],[1,5,9,1],[1,5,3,1],[1,5,10,1],[1,5,2,1],[1,5,2,1],[1,5,9,1],[1,4,8,1],[1,4,6,1],[1,4,0,1],[1,4,10,1],[1,4,5,1],[1,4,10,1],[1,4,9,1],[1,4,1,1],[1,4,4,1],[1,4,4,1],[1,4,0,1],[1,4,3,1],[1,4,1,1],[1,4,3,1],[1,4,2,1],[1,4,4,1],[1,4,4,1],[1,4,8,1],[1,4,2,1],[1,4,4,1],[1,3,2,1],[1,3,6,1],[1,3,4,1],[1,3,7,1],[1,3,4,1],[1,3,1,1],[1,3,10,1],[1,3,3,1],[1,3,4,1],[1,3,7,1],[1,3,5,1],[1,3,6,1],[1,3,1,1],[1,3,6,1],[1,3,10,1],[1,3,2,1],[1,3,4,1],[1,3,2,1],[1,3,1,1],[1,3,5,1],[1,2,4,1],[1,2,2,1],[1,2,8,1],[1,2,3,1],[1,2,1,1],[1,2,9,1],[1,2,10,1],[1,2,9,1],[1,2,4,1],[1,2,5,1],[1,2,0,1],[1,2,9,1],[1,2,9,1],[1,2,0,1],[1,2,1,1],[1,2,1,1],[1,2,4,1],[1,1,0,1],[1,1,2,1],[1,1,2,1],[1,1,5,1],[1,1,3,1],[1,1,10,1],[1,1,6,1],[1,1,0,1],[1,1,8,1],[1,1,6,1],[1,1,4,1],[1,1,9,1],[1,1,9,1],[1,1,4,1],[1,1,2,1],[1,1,9,1],[1,1,0,1],[1,1,8,1],[1,1,6,1],[1,1,1,1],[1,1,1,1],[1,1,5,1]]
num_friends = [100.0,49,41,40,25,21,21,19,19,18,18,16,15,15,15,15,14,14,13,13,13,13,12,12,11,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,8,8,8,8,8,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]

outlier = num_friends.index(100)
daily_minutes = [1,68.77,51.25,52.08,38.36,44.54,57.13,51.4,41.42,31.22,34.76,54.01,38.79,47.59,49.1,27.66,41.03,36.73,48.65,28.12,46.62,35.57,32.98,35,26.07,23.77,39.73,40.57,31.65,31.21,36.32,20.45,21.93,26.02,27.34,23.49,46.94,30.5,33.8,24.23,21.4,27.94,32.24,40.57,25.07,19.42,22.39,18.42,46.96,23.72,26.41,26.97,36.76,40.32,35.02,29.47,30.2,31,38.11,38.18,36.31,21.03,30.86,36.07,28.66,29.08,37.28,15.28,24.17,22.31,30.17,25.53,19.85,35.37,44.6,17.23,13.47,26.33,35.02,32.09,24.81,19.33,28.77,24.26,31.98,25.73,24.86,16.28,34.51,15.23,39.72,40.8,26.06,35.76,34.76,16.13,44.04,18.03,19.65,32.62,35.59,39.43,14.18,35.24,40.13,41.82,35.45,36.07,43.67,24.61,20.9,21.9,18.79,27.61,27.21,26.61,29.77,20.59,27.53,13.82,33.2,25,33.1,36.65,18.63,14.87,22.2,36.81,25.53,24.62,26.25,18.21,28.08,19.42,29.79,32.8,35.99,28.32,27.79,35.88,29.06,36.28,14.1,36.63,37.49,26.9,18.58,38.48,24.48,18.95,33.55,14.24,29.04,32.51,25.63,22.22,19,32.73,15.16,13.9,27.2,32.01,29.27,33,13.74,20.42,27.32,18.23,35.35,28.48,9.08,24.62,20.12,35.26,19.92,31.02,16.49,12.16,30.7,31.22,34.65,13.13,27.51,33.2,31.57,14.1,33.42,17.44,10.12,24.42,9.82,23.39,30.93,15.03,21.67,31.09,33.29,22.61,26.89,23.48,8.38,27.81,32.35,23.84]
# index of outlier
daily_minutes_good = [x
                      for i, x in enumerate(daily_minutes)
                      if i != outlier]
beta = least_squares_fit(inputs, daily_minutes_good, lr=10**-3, num_steps=5000, batch_size=25)
print(beta)
assert 30.5 < beta[0] < 30.70 # constant
assert 0.96 < beta[1] < 1.0 # nun friends
assert -1.89 < beta[2] < -1.85 # work hours daily
assert .91 < beta[3] < .94 # has phd


least squares fit: 100%|██████████| 5000/5000 [00:03<00:00, 1521.51it/s]

[30.51473922791934, 0.9748304586429777, -1.850689215018137, 0.9141219850728525]


### Goodness of Fit

To assess goodness of fit, we can again look at R-squared


In [6]:
def multiple_r_squared(xs: List[Vector], ys: Vector, beta: Vector) -> float:
    """
    Given the dataset and a vector beta of parameters, return the R-squared value for how well the `beta` 
    """
    explained_y_variance = total_sum_of_squares(ys)
    predicted_y_variance = total_sum_of_squares([predict(x, beta) for x in xs])
    return predicted_y_variance / explained_y_variance
assert 0.67 < multiple_r_squared(inputs, daily_minutes_good, beta) < 0.68


Unfortunately, R-squared is not a fair comparison between multiple regression and simple regression (or even multiple with less features/independent variables): because the lesser feature forms are just special cases of multiple with particular features having a coefficient of zero, the multiple regression case is necessarily equal to or better than any fewer-feature counterpart. 

The correct thing to look at, in determining whether a given feature/independent variable has value in the regression is to look at the *standard errors of the coefficients*: we assume error terms are generated from a normal distribution with mean zero and some constant unknown standard deviation. We aren't equipped to actually calc this with the pieces we have so far, but here's a wiki: [https://en.wikipedia.org/wiki/Standard_error](https://en.wikipedia.org/wiki/Standard_error)

#### Digression: The Bootstrap

Problem: we want to understand a statistic (e.g. median) about a sample, and understand how well it represents the data set as a whole. In the case that data is multi-modal, we shouldn't be as confident in a median since it is likely to switch between modes. How can we detect multi-modal-ness when we can't easily inspect the data? 

For a median, if we could continue to sample from the distribution we'd see that it varies, indicating the data isn't actually centered around a given point. For most meaningful datasets though, we can't generate new data cheaply or even at all.

**Bootstrap**: To address this, we can sub-sample our sample with replacement to generate new datasets as proxies, this being called a bootstrap. In this case, it might often make sense to generate a sub-sample equivalent in size to our sample

In [7]:
X = TypeVar('X')
Stat = TypeVar('Stat')

def bootstrap_sample(xs: List[X]) -> List[X]:
    """
    Sample a dataset with replacement to get a sub-sample
    """
    return [random.choice(xs) for _ in xs]

def bootstrap_statistic(xs: List[X], stats_fn: Callable[[List[X]], Stat], num_samples: int) -> List[Stat]:
    """
    evaluates stats_fn on num_samples bootstrap samples of xs
    """
    return [stats_fn(bootstrap_sample(xs)) for _ in range(num_samples)]

In [8]:
# Example, 101 points from normal distribution between 99.5 - 100.5
get_point: Callable[[float], float] = lambda m: random.random() + m - 0.5  # return a point very close to m, mean=m
close_to_100 = [get_point(100.) for _ in range(101)]
# 101 points, 1 near 100, 50 near 0, 50 near 200
far_from_100 = [get_point(100)] + [get_point(0) for _ in range(50)] + [get_point(200) for _ in range(50)]

In [9]:
# Note: both are very close to 100, but the datasets are very different
assert abs(median(close_to_100) - median(far_from_100)) < 1.
median(close_to_100), median(far_from_100)

(100.0456340095041, 100.37701067057527)

In [10]:
# to be fair, the global standard deviations make this obvious, but this is a toy example
assert abs(standard_deviation(close_to_100) - standard_deviation(far_from_100)) > 90.
standard_deviation(close_to_100), standard_deviation(far_from_100)

(0.3093108526176894, 100.03855781880351)

In [11]:
# we can also detect this with bootstrapping though:
medians_close = bootstrap_statistic(close_to_100, median, 100)
medians_far = bootstrap_statistic(far_from_100, median, 100)

assert standard_deviation(medians_close) < 1
assert standard_deviation(medians_far) > 90

While the above example could be discovered with manual inspection or global statistics like `standard_deviation`, presumably there are cases where this won't be true, and bootstrapping might provide insights that can't be gathered from statistics over the initial dataset

#### Standard Errors of Regression Coefficients

While we dont have the linear algebra setup to actually compute the standard errors as previously described, we can estimate them using bootstrapping:
1. Generate a bootstrapped version of of dataset (xs, ys)
2. Estimate beta using the bootstrapped data
3. For coefficients that vary significantly across bootstraps, we should lack confidence in our estimation of that parameter. For coefficients that remain close across bootstraps, we can be confident in a good estimate

In [12]:
def estimate_sample_beta(pairs: List[Tuple[Vector, float]]) -> Vector:
    """
    estimate beta for a given sample of the dataset
    """
    # xs, ys = zip(*pairs)  # split sample into xs, ys
    xs = [x for x, _ in pairs]
    ys = [y for _, y in pairs]
    beta = least_squares_fit(xs, ys, 10**-3, 5000, 25)
    print('Beta sample', beta)
    return beta

In [13]:
# for consistent results
random.seed(0)
dataset = list(zip(inputs, daily_minutes_good))
# Run 100 multiple regressions on bootstrapped data (takes a while!)
bootstrap_betas = bootstrap_statistic(dataset, estimate_sample_beta, 100)

least squares fit:   3%|▎         | 153/5000 [00:00<00:03, 1523.49it/s]

Beta sample [30.49402029547432, 1.0393791030498776, -1.9516851948558502, 0.7483721251697328]


least squares fit:   6%|▌         | 306/5000 [00:00<00:03, 1519.05it/s]

Beta sample [30.149963287526045, 1.0005300432763113, -2.0650380122822543, 3.177179854834797]


least squares fit:   3%|▎         | 152/5000 [00:00<00:03, 1511.44it/s]

...

Beta sample [31.725525991297587, 0.8939775539447468, -1.8435590604691483, -0.6224324630864055]


least squares fit:   6%|▌         | 307/5000 [00:00<00:03, 1532.66it/s]

Beta sample [30.2731194689144, 0.8005769229528958, -1.6991234036996576, 0.9748341305369913]


least squares fit: 100%|██████████| 5000/5000 [00:03<00:00, 1506.05it/s]

Beta sample [31.756965063689528, 1.0790800487199685, -2.088089407820006, 1.6420943383461708]


In [14]:
bootstrap_std_errors = [standard_deviation([beta[i] for beta in bootstrap_betas]) for i in range(4)]

In [15]:
# ~1.272 (actual is 1.19) - constant term
# ~0.103 (actual is 0.080) - num friends
# ~0.155 (actual is 0.155) - work hours
# ~1.249 (actual is .998) - phd
print(bootstrap_std_errors)
assert sum(subtract(bootstrap_std_errors, [1.27, .103, .155, 1.25])) < 0.001

[1.2715078186272788, 0.10318410116073967, 0.15510591689663633, 1.249097524805126]


###### Hyopthesis Testing and the standard error

Why compute the standard error for regression coefficients? Doing this, we can give confidence bounds for each $\beta_i$, as well as test hypotheses (e.g. does $\beta_i$ equal zero?). When $\beta_i$ represents a meaningful feature, we can start to reason about the conclusions we can draw from our data.

For example, in the above set we fit a regression model that includes a boolean term for PhD degree holders. Using the standard error of regression coefficients, we can test the hypothesis that $\beta_{PhD}$ is equal to zero, which can inform us about the relationship between having a PhD and spending time on social media represented in our data.

This makes assumptions about the distribution of $\epsilon_i$

###### How to Hypothesis Test $\beta_i ?= 0$ ?

Lets assume a null hypothesis $\beta_i = 0$. We can expect the quotient: $$t_j = \frac{\hat{\beta_i}}{\hat{\sigma_i}}$$
to be distributed following the "[Student's t-distribution](https://en.wikipedia.org/wiki/Student%27s_t-distribution)" with "[n-k](http://ordination.okstate.edu/MULTIPLE.htm) degrees of freedom". Computing this from scratch is hard, but when n (number of samples) is much larger than k (the number of fit parameters), its approximately standard normal and we can just use this instead.

So below, we'll assume $t_i \sim Normal(0, 1)$, and provide a p-value that our observed ratio between $\hat{\beta_i}$ and the standard error $\hat{\sigma_i}$ meets this expectation

In [16]:
def p_value(beta_hat_i: float, sigma_hat_i: float) -> float:
    """
    return the probabity estimate that we would observe this parameter beta given the 
    standard error if the true value of beta is zero
    """
    if beta_hat_i > 0: 
        positive_tail = 1 - normal_cdf(beta_hat_i / sigma_hat_i)  # probability that X ~ Normal(0, 1) > beta_hat_i
        return 2 * positive_tail  # we're asking about absolute deviation
    else:
        negative_tail = normal_cdf(beta_hat_i / sigma_hat_i)  # probability that X ~ Normal(0, 1) > beta_hat_i
        return 2 * negative_tail  # we're asking about absolute deviation    

In [17]:
assert p_value(beta[0], bootstrap_std_errors[0]) < 10**-3  # constant term (30.58, 1.27)
assert p_value(beta[1], bootstrap_std_errors[1]) < 10**-3  # num_friends (.972, .103)
assert p_value(beta[2], bootstrap_std_errors[2]) < 10**-3  # work_hours (-1.865, .155)
assert p_value(beta[3], bootstrap_std_errors[3]) > 0.4  # phd (.923, 1.249) 

While most of the coefficients have low p-values indicating we can confidently reject the null hypothesis and accept that they are non-zero, this is not true of the final term $\beta_{PhD}$, which is not 'significantly' different from zero. 

## Regularization

In many cases, we want to apply regression to large datasets with many features. To help with this, we may want to penalize un-needed or needlessly large $\beta$ parameters. We can regularize by adding a loss term to the function we minimize.

Where least squared error loss is:
$$ loss(x, y_i) = (\hat{y_i} - y_i)^2  = (y_i - \beta * x)^2$$

Ridge regression (where error term is the square of each $\beta$ coefficient) is minimized as follows:

$$ loss_{ridge}(x, y_i) = (\hat{y_i} - y_i)^2  + ridgepenalty = (y_i - \beta * x)^2 + \alpha\sum_{j=1}^{k}{\beta_j^2}$$

Where $\alpha$ is a hyper-parameter for regularization harshness. This means our gradient has the form:

$$ grad_{ridge}(x, y_i) = 2 * (\beta * x - y_i) * x + 2\alpha\sum_{j=1}^{k}{\beta_j}$$



This can be minimzed using gradient descent, like below:

In [18]:
def ridge_penalty(beta: Vector, alpha: float) -> float:
    """
    Return the error for x, y, beta w.r.t a ridge regression with alpha=alpha
    """
    # don't penalize the bias/constant term
    return alpha * sum(dot(beta[1:], beta[1:]))

def ridge_penalty_gradient(beta: Vector, alpha: float) -> Vector:
    """
    Return the gradient for x, y, beta w.r.t just the regularization term in a ridge regression with alpha=alpha
    """
    # don't penalize the bias/constant term in the gradient, either
    return [0] + [2 * alpha * b_i for b_i in beta[1:]]
    

def ridge_error_gradient(x: Vector, y: Vector, beta: Vector, alpha: float) -> Vector:
    """
    Return the gradient for x, y, beta w.r.t a ridge regression with alpha=alpha (just add both gradients)
    """
    return add(sq_error_gradient(x, y, beta), ridge_penalty_gradient(beta, alpha))
    

def ridge_regression_fit(xs: List[Vector], ys: List[Vector], lr:float = 10**-3, num_steps: int = 1000, batch_size: int = 1, alpha: float = 1) -> Vector:
    """
    For the given inputs (`xs`) and outputs (`ys`), find the parameters (`beta`) that provide the best fit 
    via ridge regression. Performs `num_steps` gradient descent steps with batch sizes of `batch_size` 
    and a learning rate of `lr`.
    """
    # paired = list(zip(xs, ys))
    # random.shuffle(paired)  doing this seems correct and exerimentally valid, but gets slightly worse results than the
    # book. Maybe the book has well-selected hyper parameters?
    # xs, ys = zip(*paired)
    beta = [random.random() for r in range(0, len(xs[0]))]
    for _ in tqdm.trange(num_steps, desc="ridge regression squares fit"):
        for start in range(0, len(xs), batch_size):
            # prepare a batch
            batch_xs = xs[start:start + batch_size]
            batch_ys = ys[start:start + batch_size]
            # compute an average gradient across the batch
            grads = [ridge_error_gradient(x, y, beta, alpha) for (x, y) in zip(batch_xs, batch_ys)]
            avg_grad: Vector = vector_mean(grads)
            # update beta according to the gradient
            beta = gradient_step(beta, avg_grad, lr * -1)  # by negative one b/c we are minimizing
    return beta

In [19]:
# Now run a ridge regression with alpha=0, we should get the same thing!
random.seed(0)
beta_0 = ridge_regression_fit(inputs, daily_minutes_good, lr=10**-3, num_steps=5000, batch_size=25, alpha=0)
# 30.51, 0.97, -1.85, .91
print('Ridge regression, alpha=0', beta_0)
print('Equivalent regression result:', beta)
assert 5 < dot(beta_0[1:], beta_0[1:]) < 6
assert 0.67 < multiple_r_squared(inputs, daily_minutes_good, beta_0) < 0.69


ridge regression squares fit: 100%|██████████| 5000/5000 [00:05<00:00, 936.43it/s]

Ridge regression, alpha=0 [30.51479594518558, 0.974827427732327, -1.8506912934343658, 0.9140778074476832]
Equivalent regression result: [30.51473922791934, 0.9748304586429777, -1.850689215018137, 0.9141219850728525]


In [20]:
# Now run a ridge regression with alpha=0.1, beta should be smaller, even if fit is worse
beta_0_1 = ridge_regression_fit(inputs, daily_minutes_good, lr=10**-3, num_steps=5000, batch_size=25, alpha=0.1)
# 30.8, 0.95, -1.83, .54
print('Ridge regression, alpha=0.1', beta_0_1)
print('Ridge regression, alpha=0', beta_0)
assert 4 < dot(beta_0_1[1:], beta_0_1[1:]) < 5
print(multiple_r_squared(inputs, daily_minutes_good, beta_0_1))
assert 0.66 < multiple_r_squared(inputs, daily_minutes_good, beta_0_1) < 0.67

ridge regression squares fit: 100%|██████████| 5000/5000 [00:05<00:00, 959.72it/s]

Ridge regression, alpha=0.1 [30.801525998459162, 0.95072257771587, -1.833142990416332, 0.5384447644638309]
Ridge regression, alpha=0 [30.51479594518558, 0.974827427732327, -1.8506912934343658, 0.9140778074476832]
0.6650114471578492


In [21]:
# Now run a ridge regression with alpha=1, beta should be smaller, even if fit is worse
beta_1 = ridge_regression_fit(inputs, daily_minutes_good, lr=10**-3, num_steps=5000, batch_size=25, alpha=1.)
# 30.6, 0.89, -1.67, 0.10
print('Ridge regression, alpha=1', beta_1)
print('Ridge regression, alpha=0.1', beta_0_1)
print('Ridge regression, alpha=0', beta_0)
assert 3 < dot(beta_1[1:], beta_1[1:]) < 4
print(multiple_r_squared(inputs, daily_minutes_good, beta_1))
assert 0.58 < multiple_r_squared(inputs, daily_minutes_good, beta_1) < 0.59


ridge regression squares fit: 100%|██████████| 5000/5000 [00:05<00:00, 962.61it/s]

Ridge regression, alpha=1 [30.649808578084645, 0.897152564158191, -1.6767995323859521, 0.10444589873012315]
Ridge regression, alpha=0.1 [30.801525998459162, 0.95072257771587, -1.833142990416332, 0.5384447644638309]
Ridge regression, alpha=0 [30.51479594518558, 0.974827427732327, -1.8506912934343658, 0.9140778074476832]
0.5874318952321564


In [22]:
# Now run a ridge regression with alpha=1, beta should be smaller, even if fit is worse
beta_10 = ridge_regression_fit(inputs, daily_minutes_good, lr=10**-3, num_steps=5000, batch_size=25, alpha=10.)
# 30.8, 0.95, -1.83, .54
print('Ridge regression, alpha=10', beta_10)
print('Ridge regression, alpha=1', beta_1)
print('Ridge regression, alpha=0.1', beta_0_1)
print('Ridge regression, alpha=0', beta_0)
assert 1 < dot(beta_10[1:], beta_10[1:]) < 2
print(multiple_r_squared(inputs, daily_minutes_good, beta_10))
assert 0.25 < multiple_r_squared(inputs, daily_minutes_good, beta_10) < 0.5

ridge regression squares fit: 100%|██████████| 5000/5000 [00:05<00:00, 953.83it/s]

Ridge regression, alpha=10 [28.30708308025664, 0.6726275942984856, -0.9045499907700506, -0.005213193101154039]
Ridge regression, alpha=1 [30.649808578084645, 0.897152564158191, -1.6767995323859521, 0.10444589873012315]
Ridge regression, alpha=0.1 [30.801525998459162, 0.95072257771587, -1.833142990416332, 0.5384447644638309]
Ridge regression, alpha=0 [30.51479594518558, 0.974827427732327, -1.8506912934343658, 0.9140778074476832]
0.2582567639161503


There's also lasso regression, is better at finding sparse models since its regularization drives useless coefficients to zero. Since it is not differentiable though, we can't apply gradient descent.

$$ loss_{lasso}(x, y_i) = (\hat{y_i} - y_i)^2  + lassopenalty = (y_i - \beta * x)^2 + \alpha\sum_{j=1}^{k}{|\beta_j|}$$